In [46]:
import os
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import glob

In [47]:
# 데이터 로드 및 전처리 함수
def load_and_preprocess_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
    gender = data['metadata.model']['metadata.model.gender']
    age = data['metadata.model']['metadata.model.age']
    season = data['metadata.clothes']['metadata.clothes.season']
    clothing_type = data['metadata.clothes']['clothing_type']
    
    return gender, age, season, clothing_type

# 데이터셋 폴더 경로
dataset_folder = 'Dataset/bottom'

# 데이터셋 구축
data_list = []
# Dataset/bottom 폴더의 모든 JSON 파일을 불러와서 데이터셋을 구축합니다.
for root, _, files in os.walk(dataset_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)
            gender, age, season, clothing_type = load_and_preprocess_data(json_file_path)
            data_list.append([gender, age, season, clothing_type])

# DataFrame으로 변환
df = pd.DataFrame(data_list, columns=['gender', 'age', 'season', 'clothing_type'])

# Label Encoding
label_encoders = {}

# 성별과 계절을 라벨 인코딩
for column in ['gender', 'season']:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

# 나이를 범주형으로 변환
age_categories = ['10대', '20대', '30대', '40대', '50대', '60대']
df['age'] = pd.Categorical(df['age'], categories=age_categories, ordered=True)
df = pd.get_dummies(df, columns=['age'])

# 입력 데이터와 출력 데이터 분리
X = df.drop(columns=['clothing_type']).values.astype(float)
y_clothing_type = df['clothing_type'].values
y_season = df['season'].values

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 출력 데이터 라벨 인코딩
clothing_type_encoder = LabelEncoder()
y_clothing_type = clothing_type_encoder.fit_transform(y_clothing_type)

# 학습 데이터와 검증 데이터로 분리
X_train, X_val, y_clothing_type_train, y_clothing_type_val = train_test_split(X, y_clothing_type, test_size=0.2, random_state=42)
_, _, y_season_train, y_season_val = train_test_split(X, y_season, test_size=0.2, random_state=42)


In [48]:
# DNN 모델 설계
def create_dnn_model(input_shape, output_shape_clothing_type, output_shape_season):
    inputs = Input(shape=input_shape)
    x = Dense(256, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    
    clothing_type_output = Dense(output_shape_clothing_type, activation='softmax', name='clothing_type_output')(x)
    season_output = Dense(output_shape_season, activation='softmax', name='season_output')(x)
    
    model = Model(inputs=inputs, outputs=[clothing_type_output, season_output])
    
    model.compile(
        optimizer='adam', 
        loss={
            'clothing_type_output': 'sparse_categorical_crossentropy', 
            'season_output': 'sparse_categorical_crossentropy'
        },
        metrics={
            'clothing_type_output': 'accuracy', 
            'season_output': 'accuracy'
        }
    )
    return model

# 모델 생성
model = create_dnn_model((X_train.shape[1],), len(np.unique(y_clothing_type)), len(np.unique(y_season)))

# 모델 체크포인트 콜백 설정
checkpoint = ModelCheckpoint(
    'best_model_{epoch:02d}_{val_loss:.2f}_{val_clothing_type_output_accuracy:.2f}.keras', 
    monitor='val_clothing_type_output_accuracy', 
    save_best_only=True, 
    mode='max', 
    verbose=1
)

# 얼리 스토핑 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=2, mode='min', verbose=1)

# 모델 학습
model.fit(
    X_train, 
    {'clothing_type_output': y_clothing_type_train, 'season_output': y_season_train},
    epochs=100, 
    batch_size=32,
    validation_data=(X_val, {'clothing_type_output': y_clothing_type_val, 'season_output': y_season_val}), 
    callbacks=[checkpoint, early_stopping]
)

# 가장 최근에 저장된 모델 파일 찾기
latest_model = max(glob.glob('best_model_*.keras'), key=os.path.getctime)

# 저장된 모델 로드
best_model = tf.keras.models.load_model(latest_model)

# 모델 평가
losses = best_model.evaluate(X_val, {'clothing_type_output': y_clothing_type_val, 'season_output': y_season_val}, return_dict=True)
print("Evaluation losses and metrics:", losses)

# 딕셔너리 키 확인
print("Keys in evaluation dictionary:", losses.keys())

# 각 값의 인덱스를 확인한 후 올바르게 참조
clothing_type_loss = losses.get('clothing_type_output_loss', None)
season_loss = losses.get('season_output_loss', None)
clothing_type_accuracy = losses.get('clothing_type_output_accuracy', None)
season_accuracy = losses.get('season_output_accuracy', None)

print(f'Best Clothing Type Model Accuracy: {clothing_type_accuracy:.2f}')
print(f'Best Season Model Accuracy: {season_accuracy:.2f}')

Epoch 1/100
50/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - clothing_type_output_accuracy: 0.6337 - loss: 2.0901 - season_output_accuracy: 0.4446 
Epoch 1: val_clothing_type_output_accuracy improved from -inf to 0.67255, saving model to best_model_01_1.14_0.67.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - clothing_type_output_accuracy: 0.6378 - loss: 2.0128 - season_output_accuracy: 0.4777 - val_clothing_type_output_accuracy: 0.6725 - val_loss: 1.1357 - val_season_output_accuracy: 0.9196
Epoch 2/100
56/64 ━━━━━━━━━━━━━━━━━━━━ 0s 927us/step - clothing_type_output_accuracy: 0.6190 - loss: 1.1427 - season_output_accuracy: 0.9742
Epoch 2: val_clothing_type_output_accuracy improved from 0.67255 to 0.67451, saving model to best_model_02_0.83_0.67.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - clothing_type_output_accuracy: 0.6213 - loss: 1.1278 - season_output_accuracy: 0.9757 - val_clothing_type_output_accuracy: 0.6745 - val_loss: 0.8251 - val_season_output_accuracy: 1.0000
Epoch 3/100
53/64 ━━━━

In [64]:
# 계절을 온도로 매핑하는 함수
def temperature_to_season(temperature):
    if temperature >= 25:
        return 'summer'
    elif temperature <= 5:
        return 'winter'
    else:
        return 'spring&fall'

# 예측 함수
def predict(input_data):
    input_data_transformed = [label_encoders['gender'].transform([input_data['gender']])[0]]
    
    # 나이를 원-핫 인코딩
    age_dummies = pd.get_dummies(pd.Categorical([input_data['age']], categories=age_categories))
    age_values = age_dummies.iloc[0].values
    
    season = temperature_to_season(input_data['temperature'])
    season_transformed = label_encoders['season'].transform([season])[0]
    
    input_data_array = np.concatenate([input_data_transformed, age_values, [season_transformed]]).reshape(1, -1)
    input_data_array = scaler.transform(input_data_array)
    
    predictions = best_model.predict(input_data_array)
    clothing_type_prediction = predictions[0]
    season_prediction = predictions[1]
    
    clothing_type_result = clothing_type_encoder.inverse_transform([np.argmax(clothing_type_prediction)])[0]
    season_result = label_encoders['season'].inverse_transform([np.argmax(season_prediction)])[0]
    
    return clothing_type_result, season_result

# 예측 예시
input_data = {'gender': 'FEMALE', 'age': '30대', 'temperature': 30}
clothing_type_result, season_result = predict(input_data)
print(f'Predicted Clothing Type: {clothing_type_result}')
print(f'Predicted Season: {season_result}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Clothing Type: 긴바지
Predicted Season: spring&fall
